In [1]:
import xmlrpc.client 
import pandas as pd

# Conexión con Odoo
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")




fields_info = models.execute_kw(
    db, uid, password,
    'sale.order',  # Nombre técnico del modelo
    'fields_get',
    []
)

# Convertir a DataFrame para visualizar mejor
df_campos = pd.DataFrame.from_dict(fields_info, orient='index')
df_campos

,sortable,help,searchable,required,manual,readonly,depends,groups,company_dependent,translate,...,domain,context,states,relation_field,selection,digits,size,change_default,related,sanitize
origin,True,Reference of the document that generated this ...,True,False,False,False,[],False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
message_follower_ids,False,NaN,True,False,False,False,[],False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
create_date,True,Date on which sales order is created.,True,False,False,True,[],False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
store_id,True,Sucursal.,True,True,False,False,[],False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
categ_ids,True,NaN,True,False,False,False,[],False,False,NaN,...,"['|', ('section_id', '=', section_id), ('secti...",{'object_name': 'crm.lead'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
write_date,True,NaN,True,False,False,False,[],False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
website_order_line,True,Order Lines to be displayed on the website. Th...,True,False,False,True,[],False,False,NaN,...,NaN,NaN,NaN,order_id,NaN,NaN,NaN,NaN,NaN,NaN
website_message_ids,True,Website communication history,True,False,False,False,[],False,False,NaN,...,"[&, [model, =, sale.order], [type, =, comment]]",NaN,NaN,res_id,NaN,NaN,NaN,NaN,NaN,NaN
product_id,False,NaN,True,False,False,False,[],False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# TOTAL FACTURAS 

In [1]:
import xmlrpc.client 
from datetime import date
import pandas as pd

# Conexión con Odoo
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos


# --- Autenticación ---
common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# --- Fechas ---
from datetime import date, timedelta , datetime

weeks = 52

# --- Fechas automáticas ---
hoy = date.today()
fecha_fin = hoy.strftime("%Y-%m-%d")
fecha_inicio = (hoy - timedelta(weeks=weeks)).strftime("%Y-%m-%d")

# --- 1) Buscar facturas válidas ---

invoice_domain = [
    ("date_invoice", ">=", fecha_inicio),
    ("date_invoice", "<=", fecha_fin),
    ("type", "=", "out_invoice"),   # solo ventas
    ("state", "in", ["open", "paid"])
]

invoice_ids = models.execute_kw(
    db, uid, password,
    "account.invoice", "search",
    [invoice_domain]
)
print(f"Facturas encontradas: {len(invoice_ids)}")

# --- 2) Descargar las líneas de esas facturas ---
fields = ["product_id", "quantity", "price_subtotal", "invoice_id","create_date","store"] 

records = []
limit = 20000
offset = 0

while True:
    result = models.execute_kw(
        db, uid, password,
        "account.invoice.line", "search_read",
        [[("invoice_id", "in", invoice_ids)]],
        {"fields": fields, "limit": limit, "offset": offset}
    )
    if not result:
        break
    records.extend(result)
    offset += limit
    print(f"Descargados {len(records)} registros...")

# --- 3) Pasar a DataFrame ---
df = pd.DataFrame(records).fillna(0)

# --- 3) Pasar a DataFrame ---
df = pd.DataFrame(records).fillna(0)

# Separar product_id
df["product_id_num"] = df["product_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
df["product_name"] = df["product_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar invoice_id
df["invoice_id_num"] = df["invoice_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
df["invoice_name"] = df["invoice_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

#df["company_id"] = df["company_id"].astype(str)

# Convertir fecha a datetime
df["date_invoice"] = pd.to_datetime(df["create_date"], errors="coerce")

# Eliminar las columnas originales problemáticas
df = df.drop(columns=["product_id", "invoice_id","create_date"])


df.to_parquet(f"ventashistoricas{weeks}semanas.parquet", index=False)

print(f"Total de líneas descargadas: {len(df)}")

#para 52 semanas se demora aproximadamente 6 minutos

Facturas encontradas: 51042
Descargados 20000 registros...
Descargados 40000 registros...
Descargados 60000 registros...
Descargados 80000 registros...
Descargados 100000 registros...
Descargados 120000 registros...
Descargados 140000 registros...
Descargados 160000 registros...
Descargados 180000 registros...
Descargados 198432 registros...
Total de líneas descargadas: 198432


In [3]:
print(f"Fecha primera factura",df["date_invoice"].min())
print(f"Fecha ultima factura",df["date_invoice"].max())

Fecha primera factura 2024-08-20 11:59:55
Fecha ultima factura 2025-09-15 19:50:53


# FACTURAS ULTIMAS 12 SEMANAS

In [2]:
import xmlrpc.client 
from datetime import date
import pandas as pd

# Conexión con Odoo
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos


# --- Autenticación ---
common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# --- Fechas ---
from datetime import date, timedelta

# --- Fechas automáticas ---
hoy = date.today()
fecha_fin = hoy.strftime("%Y-%m-%d")
fecha_inicio = (hoy - timedelta(weeks=12)).strftime("%Y-%m-%d")
# --- 1) Buscar facturas válidas ---

invoice_domain = [
    ("date_invoice", ">=", fecha_inicio),
    ("date_invoice", "<=", fecha_fin),
    ("type", "=", "out_invoice"),   # solo ventas
    ("state", "in", ["open", "paid"])
]

invoice_ids = models.execute_kw(
    db, uid, password,
    "account.invoice", "search",
    [invoice_domain]
)
print(f"Facturas encontradas: {len(invoice_ids)}")

# --- 2) Descargar las líneas de esas facturas ---
fields = ["product_id", "quantity", "price_subtotal", "invoice_id","create_date","company_id","store"] #company id no va siempre muestra donsson

records = []
limit = 20000
offset = 0

while True:
    result = models.execute_kw(
        db, uid, password,
        "account.invoice.line", "search_read",
        [[("invoice_id", "in", invoice_ids)]],
        {"fields": fields, "limit": limit, "offset": offset}
    )
    if not result:
        break
    records.extend(result)
    offset += limit
    print(f"Descargados {len(records)} registros...")

# --- 3) Pasar a DataFrame ---
df = pd.DataFrame(records).fillna(0)

# --- 3) Pasar a DataFrame ---
df = pd.DataFrame(records).fillna(0)

# Separar product_id
df["product_id_num"] = df["product_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
df["product_name"] = df["product_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar invoice_id
df["invoice_id_num"] = df["invoice_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
df["invoice_name"] = df["invoice_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

df["company_id"] = df["company_id"].astype(str)

# Convertir fecha a datetime
df["date_invoice"] = pd.to_datetime(df["create_date"], errors="coerce")

# Eliminar las columnas originales problemáticas
df = df.drop(columns=["product_id", "invoice_id","create_date"])


df.to_parquet("historico3mesesEMA.parquet", index=False)

print(f"Total de líneas descargadas: {len(df)}")

Facturas encontradas: 12171
Descargados 20000 registros...
Descargados 40000 registros...
Descargados 47007 registros...
Total de líneas descargadas: 47007


# COSTO DEL PRODUCTO

In [3]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

campos = models.execute_kw(
    db , uid , password,
    "cs.analisis.costo","fields_get",
    [],

)

df_campos = pd.DataFrame(campos)
#df_campos.info()


productos_costo = models.execute_kw(
    db , uid , password,
    "cs.analisis.costo","search_read",
    [[]],
    {"fields":["producto_costo_unitario","producto_id"]}


)

# Convertir en df
df_productos_costo = pd.DataFrame(productos_costo)


# Separar lista anidada
df_productos_costo[["product_id_num","product_name"]] = (df_productos_costo["producto_id"].apply(pd.Series))

#Agregar columna estandarizada
df_productos_costo["product_name"] = df_productos_costo["product_name"].str.strip()

#DF normalizado
df_productos_costo = df_productos_costo[["product_name","producto_costo_unitario"]]

df_productos_costo.to_parquet("costo_productos.parquet")


# ANALISIS COSTOS

In [14]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

productos_ema = models.execute_kw(
    db, uid, password,
    "cs.analisis.costo", "search_read",[],
    #[
    #    [["semana","=",36]]
    #],
    {"fields": ["sucursal_id","almacen_id","product_name","ano","semana","ema","producto_costo_unitario" ]}
)

df_campos = pd.DataFrame(productos_ema)
df_campos.to_csv("real_ema.csv")

#timpo aprox  1 minutos

# VENTAS PERDIDAS

In [ ]:
import xmlrpc.client 
from datetime import date
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# ===============================
# Leer ventas perdidas desde 2025
# ===============================

ventas_perdidas_ids = models.execute_kw(
    db, uid, password,
    "ventas.perdidas", "search",
    [[["fecha", ">=", "2024-12-01"]], ["almacenamiento_tipo","=","agotado"]] 
)

ventas_perdidas = models.execute_kw(
    db, uid, password,
    "ventas.perdidas", "read",
    [ventas_perdidas_ids],
    {"fields": ["origen", "store_id", "warehouse_id", "cliente_id",
                "fecha", "product_ref", "product_id",
                "cantidad", "cantidad_existencia","cantidad_reservada","almacenamiento_tipo"]}
)

df = pd.DataFrame(ventas_perdidas)

# ===============================
# Normalizar columnas con tuplas (many2one)
# ===============================
def extract_id(val):
    if isinstance(val, (list, tuple)) and len(val) > 0:
        return val[0]
    return None

def extract_name(val):
    if isinstance(val, (list, tuple)) and len(val) > 1:
        return val[1]
    return str(val)

df["product_id_num"] = df["product_id"].apply(extract_id)
df["product_name"]   = df["product_id"].apply(extract_name)

df["cliente_id_num"] = df["cliente_id"].apply(extract_id)
df["cliente_name"]   = df["cliente_id"].apply(extract_name)

df["store_id_num"] = df["store_id"].apply(extract_id)
df["store_name"]   = df["store_id"].apply(extract_name)

df["warehouse_id_num"] = df["warehouse_id"].apply(extract_id)
df["warehouse_name"]   = df["warehouse_id"].apply(extract_name)

df = df.drop(["product_id_num","store_id","warehouse_id","cliente_id","id","cliente_id_num","warehouse_id_num","warehouse_name"],axis=1) #eliminar columnas que no sirven

# ===============================
# Forzar tipos seguros
# ===============================
for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].astype("string")

# ===============================
# Exportar a Parquet
# ===============================
try:
    df.to_parquet("ventas_perdidas_2025.parquet", index=False, engine="pyarrow", compression="snappy")
    print("✅ Exportación exitosa con pyarrow")
except Exception as e:
    print("⚠️ pyarrow falló, probando con fastparquet:", e)
    df.to_parquet("ventas_perdidas_2025.parquet", index=False, engine="fastparquet", compression="snappy")
    print("✅ Exportación exitosa con fastparquet")
    

    ##TARDA MAS O MENOS 5 MINUTOS TODO 2025


✅ Exportación exitosa con pyarrow


# MOTOR BUSQUEDA

In [3]:
import xmlrpc.client
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# ===============================
# Ver campos disponible
# ===============================
fields = models.execute_kw(
    db, uid, password,
    "cs.analisis.costo", "fields_get",
    []
)

for field, info in sorted(fields.items(), key=lambda x: x[1].get("string", "")):
    print(f"{field:25} | {info.get('type')} | {info.get('string')}")



acumulado_cantidad        | float | Acumulado cantidad
acumulado_combinado       | float | Acumulado combinado
acumulado_costo           | float | Acumulado costo
acumulado_desviacion_p    | float | Acumulado desviacion porcentaje
almacen_id                | many2one | Almacen
ano                       | char | Año
clasificacion             | char | Clasificacion
producto_code             | char | Codigo producto
producto_costo_unitario   | float | Costo unitario
create_uid                | many2one | Created by
create_date               | datetime | Created on
producto_id               | many2one | Descripcion
despacho                  | integer | Despacho
desviacion_ema            | float | Desviacion EMA
desviacion_ema_p          | float | Desviacion EMA porcentaje
display_name              | char | Display Name
ema                       | float | Ema
estado_despacho           | selection | Estado despacho
id                        | integer | Id
inventario_seguridad      | float | 

In [15]:
import xmlrpc.client
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# 1) Traer nóminas (slips) solo de 2025
# ========================================
slips = models.execute_kw(
    db, uid, password,
    "hr.payslip", "search_read",
    [[["date_from", ">=", "2024-01-01"], ["date_to", "<=", "2025-12-31"]]],
    {"fields": ["id", "employee_id", "name", "date_from", "date_to"]}
)
df_slips = pd.DataFrame(slips)


# ========================================
# 2) Traer líneas de nómina (solo de los slips obtenidos)
# ========================================
slip_ids = df_slips["id"].tolist()
lines = models.execute_kw(
    db, uid, password,
    "hr.payslip.line", "search_read",
    [[["slip_id", "in", slip_ids]]],
    {"fields": ["id", "slip_id", "employee_id", "salary_rule_id", "total"]}
)
df_lines = pd.DataFrame(lines)

# ===============================
# 3) Expandir campos Many2one (id, name)
# ===============================
def extract_name(val):
    if isinstance(val, list) and len(val) == 2:
        return val[1]
    return None

def extract_id(val):
    if isinstance(val, list) and len(val) == 2:
        return val[0]
    return None

df_lines["employee_name"] = df_lines["employee_id"].apply(extract_name)
df_lines["slip_name"]     = df_lines["slip_id"].apply(extract_name)
df_lines["rule_name"]     = df_lines["salary_rule_id"].apply(extract_name)

# 🔑 Extraer ID de slip_id para poder hacer el merge
df_lines["slip_id_only"] = df_lines["slip_id"].apply(extract_id)

# ===============================
# 4) Unir líneas con fechas de slip
# ===============================
df_slips["employee_name"] = df_slips["employee_id"].apply(extract_name)

df = pd.merge(
    df_lines,
    df_slips[["id", "employee_name", "date_from", "date_to"]],
    left_on="slip_id_only",
    right_on="id",
    how="left",
    suffixes=("", "_slip")
)

# ===============================
# 5) Filtrar reglas relevantes
# ===============================
df_filtro = df[df["rule_name"].isin(
    ["Salario Básico", "Salario Mínimo", "Salario Neto"]
)].copy()

# ===============================
# 6) Agregar columna período (Año-Mes)
# ===============================
df_filtro["periodo"] = pd.to_datetime(df_filtro["date_from"]).dt.to_period("M")

# ===============================
# 7) Pivotear para resumir salarios
# ===============================
tabla = df_filtro.pivot_table(
    index=["employee_name", "periodo"],
    columns="rule_name",
    values="total",
    aggfunc="sum",
    fill_value=0
).reset_index()

# ===============================
# 8) Renombrar columnas
# ===============================
tabla = tabla.rename(columns={
    "Salario Básico": "salario_basico",
    "Salario Mínimo": "salario_minimo",
    "Salario Neto": "salario_neto"
})

# ===============================
# 9) Calcular salario base = máximo entre básico y mínimo
# ===============================
tabla["salario_base"] = tabla[["salario_basico", "salario_minimo"]].max(axis=1)

# ===============================
# 10) Tabla final
# ===============================
resultado = tabla[["employee_name", "periodo", "salario_base", "salario_neto"]]

print("✅ Resumen mensual de salarios")

#aprox2min

✅ Resumen mensual de salarios


In [18]:
r =resultado.sort_values(["salario_neto"],ascending=False)
r = r[r["periodo"]=="2025-09"]
r.head(30)

rule_name,employee_name,periodo,salario_base,salario_neto
927,GERMAN RICARDO BETANCOURT GUTIERREZ,2025-09,1423500.0,15551900.0
1364,JUAN IGNACIO NIETO NEIVA,2025-09,3600000.0,13414292.0
1023,JAIME ALBERTO CAÑON RAMIREZ,2025-09,1423500.0,7492260.0
906,GERMAN ANTONIO MORALES CARDENAS,2025-09,1423500.0,6936680.0
1266,JUAN CARLOS BETANCOURT GUTIERREZ,2025-09,4400000.0,6238869.0
1870,OMAR YADIR HERNANDEZ PULGARIN,2025-09,1423500.0,5736458.0
1485,LEONARDO MORA CUBILLOS,2025-09,1423500.0,5316042.0
1131,JESUS ADONAY GONZALEZ FLOREZ,2025-09,1450000.0,5081168.0
2068,SIMON BETANCOURT DEL PINO,2025-09,5350000.0,4921800.0
960,HUGO ALEJANDRO ZUBIETA FERNANDEZ,2025-09,1423500.0,4903609.0
